The Rocket model from sktime library does not contain an implementation of the predic_proba method, because based on the Ridge classifier. Therefore, in this notebook, we will apply data transform using the Rocket transformer, and then on the transformed data we will try other classifiers that contain the implementation of the predict_proba method. 

In [52]:
# %pip install sktime
# %pip install numba
#%pip install dask[dataframe] --upgrade
# %pip install onnx
# %pip install onnxruntime

In [53]:
import pandas as pd
import numpy as np
from tqdm import tqdm

# from sktime.base import load
import pickle

In [54]:
def read_initial_csv(path_to_file:str, columns_list):

    separator = ' '
    data = pd.read_csv(path_to_file, nrows=1)
    if len(data.loc[0,columns_list[0]].split(',')) > 1:
        separator = ','
    
    return pd.read_csv(
        path_to_file,
        converters=dict.fromkeys(
            columns_list,
            lambda s: np.fromstring(s[1:-1], sep=separator, dtype=float)
        )
    )


def prepare_data(debug, n_samples=10_000):
    from sklearn.preprocessing import StandardScaler
    from sklearn.model_selection import train_test_split
    

    negative_data = pd.read_parquet('../data/light-curves/negative_class_iterpolator.parquet')
    positive_data = pd.read_parquet('../data/light-curves/positive_class_iterpolator.parquet')
    negative_data = negative_data.sample(n=positive_data.shape[0], random_state=42)

    if DEBUG:
        negative_data = negative_data.sample(n=n_samples, random_state=42)
        positive_data = positive_data.sample(n=n_samples, random_state=42)
        
    negative_data['class'] = 0
    positive_data['class'] = 1
    data = pd.concat([negative_data, positive_data])
    
    features_list = data.columns.drop(['id_obs','class'])
    x = data.loc[:,features_list]
    y = data['class']

    # scaling data
    scaler = StandardScaler()
    x = scaler.fit_transform(x)

    # split data
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)
    # convert to 3D np.array for sktime format data
    X_train = X_train[:,np.newaxis, :]
    X_test = X_test[:,np.newaxis, :]
    
    return X_train, X_test, y_train, y_test, scaler


def get_validation_data(initial_data, scaler, n_grid, verbose=0) -> pd.DataFrame:
    """Return dataframe with interpolated by 
    Akima1DInterpolator for n_grid points."""
    from sklearn.preprocessing import StandardScaler
    from sklearn.model_selection import train_test_split
    from scipy.interpolate import Akima1DInterpolator


    columns=(
        ['is_flare','mag_min'] 
        + [f'mag_{i}' for i in range(n_grid)]
        + ['magerr_mean', 'magerr_std']
    )

    intertpolate_data = []

    for index, item in tqdm(initial_data.iterrows(), total=initial_data.shape[0], desc='Interpolate progress'):

        row = []
        row.append(int(item["is_flare"]))
        # interpolate magnitude data
        x = item['mjd']
        y = item['mag']
        # subtract a minimum magnitude for normalize
        y_min = y.min()
        y = y - y_min
        row+=[y_min]
        # check input data
        if len(x) != len(y):
            if verbose == 1 and len(x) != len(y):
                print(
                    f'len time points={len(x)} not equl len mag points={len(y)}'
                    f' in data for obs_id={row[0]} object'
                )
                continue
        interpolator = Akima1DInterpolator(x, y)
        xnew = np.linspace(x.min(), x.max(), n_grid)
        ynew = interpolator(xnew)
        # add features to output list
        row+= list(ynew)

        # interpolate magerror data
        errors = item['magerr']
        errors_mean = errors.mean()
        errors_std = errors.std()
        row+=[errors_mean, errors_std]

        intertpolate_data.append(row)

    data = pd.DataFrame(data=intertpolate_data, columns=columns)

    x = data.drop(['is_flare'], axis=1)
    y = data['is_flare']

    # scaling data
    x = scaler.transform(x)
    x = x[:,np.newaxis, :]

    return x, y


def metrics_calc(df, model_name, y, predict):
    from sklearn.metrics import (
        accuracy_score, roc_auc_score, f1_score, precision_score,
        fbeta_score, recall_score
    )
    df.loc[model_name,:] = [
        accuracy_score(y, predict),
        roc_auc_score(y, predict),
        f1_score(y, predict),
        precision_score(y, predict),
        sum(predict),
        recall_score(y, predict),
        fbeta_score(y, predict, beta=0.1),
        fbeta_score(y, predict, beta=0.3),
        fbeta_score(y, predict, beta=0.5)
    ]
    return df

def get_metric_threshold(y_pred_proba, y_val):
    
    df_metric = pd.DataFrame(columns=[
        'accuracy', 'ROC_AUC', 'F1 score', 'precision', 'count_objects', 'recall',
        'f-beta score (beta=0.1)','f-beta score (beta=0.3)','f-beta score (beta=0.5)'
    ])
    
    for t in np.arange(0.1,1.0, 0.1):
        threshold = 1 - t
        y_pred = np.where(y_pred_proba[:,1]>=threshold, 1, 0)
        df_metric = metrics_calc(df_metric, f'Rocket_validation_threshold={threshold:.1f}', y_val, y_pred)
    
    return df_metric

# 1. Transform data by Rocket

## 1.1 Learning data

In [55]:
from sktime.transformations.panel.rocket import MiniRocket

In [56]:
DEBUG = False
X_train, X_test, y_train, y_test, scaler = prepare_data(DEBUG, n_samples=1_000)

pickle.dump(scaler, open(f'scaler debug - {DEBUG}.pkl','wb'))

In [57]:
#!c1.8
%%time
mini_rocket = MiniRocket(random_state=42, n_jobs=8)  # by default, MiniRocket uses 10,000 kernels
mini_rocket.fit(X_train)
# mini_rocket.save(f'Rocket_transform debug - {DEBUG}')
pickle.dump(mini_rocket, open(f'Rocket_transform debug - {DEBUG}.pkl','wb'))

CPU times: user 1.51 s, sys: 966 ms, total: 2.47 s
Wall time: 19.4 s


In [58]:
#!c1.32
%%time
X_train_transform = mini_rocket.transform(X_train)
X_test_transform = mini_rocket.transform(X_test)

CPU times: user 14min 12s, sys: 1min 56s, total: 16min 9s
Wall time: 10min 22s


## 1.2 Validation data

In [59]:
scaler = pickle.load(open(f'scaler debug - {DEBUG}.pkl','rb'))
mini_rocket_transform = pickle.load(open(f'Rocket_transform debug - {DEBUG}.pkl','rb'))
    
data = read_initial_csv('raw_real_flares_test.csv',['mjd','mag','magerr'])

x_val, y_val = get_validation_data(data, n_grid=100, scaler=scaler, verbose=1)
x_val = mini_rocket_transform.transform(x_val)

Interpolate progress: 100%|██████████| 1102/1102 [00:00<00:00, 2679.47it/s]


# 2. LogisticRegression

## 2.1 Learning


In [60]:
#!c1.32
%%time
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression(solver='sag', max_iter=100, random_state=42, n_jobs=32)
classifier.fit(X_train_transform, y_train)

pickle.dump(classifier, open(f'LogisticRegression classifier debug - {DEBUG}.pkl','wb'))

y_test_proba = classifier.predict_proba(X_test_transform)
y_test_pred = classifier.predict(X_test_transform)
y_train_pred = classifier.predict(X_train_transform)
print("LogRegression classifier")
df_metric = get_metric_threshold(y_test_proba, y_test)
df_metric.head(10)

/home/jupyter/.local/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


LogRegression classifier
CPU times: user 1h 32min 53s, sys: 4min 23s, total: 1h 37min 17s
Wall time: 1h 46min 3s


,accuracy,ROC_AUC,F1 score,precision,count_objects,recall,f-beta score (beta=0.1),f-beta score (beta=0.3),f-beta score (beta=0.5)
Rocket_validation_threshold=0.9,0.925585,0.925625,0.920129,0.993587,90600,0.856785,0.992019,0.980659,0.96284
Rocket_validation_threshold=0.8,0.95075,0.950772,0.948804,0.988449,96963,0.912217,0.987632,0.981676,0.9722
Rocket_validation_threshold=0.7,0.960335,0.960349,0.959417,0.982753,100192,0.937163,0.98228,0.978821,0.973284
Rocket_validation_threshold=0.6,0.964659,0.964666,0.964254,0.976015,102564,0.952773,0.975779,0.974053,0.971276
Rocket_validation_threshold=0.5,0.966168,0.96617,0.966107,0.968422,104565,0.963804,0.968376,0.968039,0.967494
Rocket_validation_threshold=0.4,0.964559,0.964555,0.96484,0.957787,106625,0.971999,0.957925,0.958944,0.960596
Rocket_validation_threshold=0.3,0.959697,0.959686,0.960475,0.94282,109076,0.978804,0.943163,0.94569,0.949803
Rocket_validation_threshold=0.2,0.948912,0.948891,0.950724,0.918654,112667,0.985114,0.919268,0.9238,0.931219
Rocket_validation_threshold=0.1,0.92059,0.920549,0.925877,0.868518,119925,0.991348,0.869585,0.877495,0.890587


## 2.2 Validation 

In [61]:
import warnings
warnings.filterwarnings("ignore")

# scaler = pickle.load(f'scaler debug - {DEBUG}.pkl')
# mini_rocket_transform = load(f'Rocket_transform debug - {DEBUG}')
# mini_rocket_transform = pickle.load(f'Rocket_transform debug - {DEBUG}.pkl')
clf = pickle.load(open(f'LogisticRegression classifier debug - {DEBUG}.pkl','rb'))
    
# data = read_initial_csv('raw_real_flares_test.csv',['mjd','mag','magerr'])

# x_val, y_val = get_validation_data(data, n_grid=100, scaler=scaler, verbose=1)
# x_val = mini_rocket_transform.transform(x_val)

print(f"Balance classes: {data.value_counts('is_flare')}")

y_pred_proba = clf.predict_proba(x_val)

print('Metrics on validattion LogRegression classifier')
df_metric = get_metric_threshold(y_pred_proba, y_val)
df_metric.head(10)

Balance classes: is_flare
0.0    1000
1.0     102
dtype: int64
Metrics on validattion LogRegression classifier


,accuracy,ROC_AUC,F1 score,precision,count_objects,recall,f-beta score (beta=0.1),f-beta score (beta=0.3),f-beta score (beta=0.5)
Rocket_validation_threshold=0.9,0.967332,0.849941,0.8,0.923077,78,0.705882,0.920273,0.900206,0.869565
Rocket_validation_threshold=0.8,0.967332,0.876353,0.8125,0.866667,90,0.764706,0.865524,0.857229,0.844156
Rocket_validation_threshold=0.7,0.963702,0.900765,0.807692,0.792453,106,0.823529,0.792749,0.79493,0.798479
Rocket_validation_threshold=0.6,0.960073,0.898765,0.792453,0.763636,110,0.823529,0.764187,0.76825,0.774908
Rocket_validation_threshold=0.5,0.949183,0.897167,0.752212,0.685484,124,0.833333,0.68669,0.695675,0.710702
Rocket_validation_threshold=0.4,0.936479,0.894569,0.710744,0.614286,140,0.843137,0.615941,0.628368,0.649547
Rocket_validation_threshold=0.3,0.926497,0.893471,0.682353,0.568627,153,0.852941,0.57051,0.584721,0.609244
Rocket_validation_threshold=0.2,0.909256,0.892775,0.640288,0.505682,176,0.872549,0.507796,0.523869,0.552109
Rocket_validation_threshold=0.1,0.857532,0.873078,0.536873,0.383966,237,0.892157,0.386144,0.402917,0.433333


# 3. Random forest

## 3.1 Learing


In [62]:
#!c1.32
%%time
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(max_depth=8, random_state=42, n_jobs=32)
classifier.fit(X_train_transform, y_train)

pickle.dump(classifier, open(f'RandomForest classifier debug - {DEBUG}.pkl','wb'))

y_test_proba = classifier.predict_proba(X_test_transform)
y_test_pred = classifier.predict(X_test_transform)
y_train_pred = classifier.predict(X_train_transform)
print("RandomForest classifier")
df_metric = get_metric_threshold(y_test_proba, y_test)
df_metric.head(10)

RandomForest classifier
CPU times: user 1h 11min 53s, sys: 5min 18s, total: 1h 17min 12s
Wall time: 18min 34s


,accuracy,ROC_AUC,F1 score,precision,count_objects,recall,f-beta score (beta=0.1),f-beta score (beta=0.3),f-beta score (beta=0.5)
Rocket_validation_threshold=0.9,0.722191,0.72235,0.616505,0.996325,47069,0.446348,0.984316,0.90432,0.79934
Rocket_validation_threshold=0.8,0.84807,0.848153,0.822604,0.98905,74797,0.70411,0.985103,0.957071,0.914994
Rocket_validation_threshold=0.7,0.898453,0.8985,0.889425,0.97689,87798,0.816334,0.974992,0.961279,0.939918
Rocket_validation_threshold=0.6,0.91929,0.919314,0.915749,0.958374,96118,0.876754,0.957492,0.951064,0.940856
Rocket_validation_threshold=0.5,0.92619,0.926195,0.925573,0.933918,103205,0.917376,0.933751,0.932529,0.930562
Rocket_validation_threshold=0.4,0.918733,0.918717,0.92094,0.897075,110809,0.94611,0.897536,0.900931,0.906471
Rocket_validation_threshold=0.3,0.893787,0.893743,0.901356,0.841814,121060,0.969962,0.842917,0.851098,0.864661
Rocket_validation_threshold=0.2,0.83129,0.8312,0.854145,0.75257,137853,0.987417,0.754346,0.767645,0.790156
Rocket_validation_threshold=0.1,0.688359,0.688181,0.762161,0.616447,170112,0.998087,0.618789,0.636544,0.667493


## 3.2 Validation

In [63]:
# scaler = pickle.load(f'scaler debug - {DEBUG}.pkl')
# mini_rocket_transform = load(f'Rocket_transform debug - {DEBUG}')
# mini_rocket_transform = pickle.load(f'Rocket_transform debug - {DEBUG}.pkl')
clf = pickle.load(open(f'RandomForest classifier debug - {DEBUG}.pkl','rb'))

# data = read_initial_csv('raw_real_flares_test.csv',['mjd','mag','magerr'])

# x_val, y_val = get_validation_data(data, n_grid=100, scaler=scaler, verbose=1)
# x_val = mini_rocket_transform.transform(x_val)

print(f"Balance classes: {data.value_counts('is_flare')}")
                  
y_pred_proba = clf.predict_proba(x_val)

print('Metrics on validattion RandomForest classifier')
df_metric = get_metric_threshold(y_pred_proba, y_val)
df_metric.head(10)

Balance classes: is_flare
0.0    1000
1.0     102
dtype: int64
Metrics on validattion RandomForest classifier


,accuracy,ROC_AUC,F1 score,precision,count_objects,recall,f-beta score (beta=0.1),f-beta score (beta=0.3),f-beta score (beta=0.5)
Rocket_validation_threshold=0.9,0.91833,0.558824,0.210526,1.0,12,0.117647,0.930876,0.617564,0.4
Rocket_validation_threshold=0.8,0.935572,0.709186,0.553459,0.77193,57,0.431373,0.765943,0.72469,0.666667
Rocket_validation_threshold=0.7,0.948276,0.826235,0.707692,0.741935,93,0.676471,0.741225,0.736054,0.727848
Rocket_validation_threshold=0.6,0.935572,0.858853,0.687225,0.624,125,0.764706,0.625139,0.633626,0.647841
Rocket_validation_threshold=0.5,0.912886,0.872765,0.636364,0.518519,162,0.823529,0.520427,0.534876,0.56
Rocket_validation_threshold=0.4,0.876588,0.861569,0.558442,0.417476,206,0.843137,0.419573,0.435635,0.464363
Rocket_validation_threshold=0.3,0.826679,0.856078,0.487936,0.335793,271,0.892157,0.33788,0.354022,0.383642
Rocket_validation_threshold=0.2,0.700544,0.799784,0.362934,0.225962,416,0.921569,0.227663,0.24098,0.266138
Rocket_validation_threshold=0.1,0.435572,0.675794,0.241463,0.137883,718,0.970588,0.139064,0.148395,0.166443
